# interpretation of drowsy detection

process方法实现了疲劳检测的核心算法
1. 处理输入帧：通过mp处理当前的视频帧，提取面部的关键点
2. 计算EAR：计算眼睛的EAR指标以检测疲劳状态
3. 管理疲劳时间：累计疲劳时间，并在时间超过设定值的时候播放警报
4. 动态更新状态：根据EAR指标更新状态和界面信息
5. 可视化信息：在视频帧上绘制与检测相关的文本和眼睛关键点